In [1]:
%%capture

!curl https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release | tar xz
%cd sqlite/
!./configure
!make sqlite3.c
%cd /content
!npx degit coleifer/pysqlite3 -f
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build
!cp build/lib.linux-x86_64-3.7/pysqlite3/_sqlite3.cpython-37m-x86_64-linux-gnu.so \
     /usr/lib/python3.7/lib-dynload/

Среда выполнения - Перезапустить среду выполнения

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.35.2'

In [5]:
import pandas as pd
import numpy as np

In [6]:
con = sqlite3.connect('db')
cur = con.cursor()
def select(sql):
  return pd.read_sql(sql,con)

### Increment

In [9]:
sql = '''with rec(x) as (
select 1
union all
select t.x + 1 from rec t
where t.x < 5
)
select * from rec t'''

In [10]:
select(sql)

,x
0,1
1,2
2,3
3,4
4,5


### Dates

In [11]:
sql = '''with dates(dt) as (
select date('2021-03-01')
union all
select date(t.dt,'+1 month') as dt from dates t
limit 5
)
select * from dates t'''

In [12]:
select(sql)

,dt
0,2021-03-01
1,2021-04-01
2,2021-05-01
3,2021-06-01
4,2021-07-01


### Fibonacci number

In [14]:
sql = '''with fib(n1,n2,n3) as (
select null,1,1
union all
select 
t.n2 as n1, 
t.n3 as n2,
t.n2 + t.n3 as n3
from fib t
limit 5
)
select * from fib t'''

In [15]:
select(sql)

,n1,n2,n3
0,NaN,1,1
1,1.0,1,2
2,1.0,2,3
3,2.0,3,5
4,3.0,5,8


### Running total

In [16]:
df = pd.DataFrame({'id':[1,2,3,4,5,6],'num':[1,5,4,9,14,27]})
df.to_sql('seq',con,index=False,if_exists='replace')

In [39]:
sql = '''with running(id,num,rt) as (
select t.id,t.num, t.num  from seq t
where t.id = 1
union all
select s.id,s.num, t.rt + s.num as rt from running t
join seq s on t.id = s.id - 1
limit 10
)
select * from running t'''

In [40]:
select(sql)

,id,num,rt
0,1,1,1
1,2,5,6
2,3,4,10
3,4,9,19
4,5,14,33
5,6,27,60


### Report to manager

In [41]:
sql = '''
drop table if exists employees;
Create table If Not Exists employees (employee_id int, employee_name varchar(30), manager_id int)
insert into Employees (employee_id, employee_name, manager_id) values ('1', 'Boss', '1')
insert into Employees (employee_id, employee_name, manager_id) values ('3', 'Alice', '3')
insert into Employees (employee_id, employee_name, manager_id) values ('2', 'Bob', '1')
insert into Employees (employee_id, employee_name, manager_id) values ('4', 'Daniel', '2')
insert into Employees (employee_id, employee_name, manager_id) values ('7', 'Luis', '4')
insert into Employees (employee_id, employee_name, manager_id) values ('8', 'John', '3')
insert into Employees (employee_id, employee_name, manager_id) values ('9', 'Angela', '8')
insert into Employees (employee_id, employee_name, manager_id) values ('77', 'Robert', '1')'''

In [42]:
sql = ';\n'.join(sql.split('\n'))

In [43]:
cur.executescript(sql)

In [44]:
sql = '''select * from employees t'''

In [45]:
select(sql)

,employee_id,employee_name,manager_id
0,1,Boss,1
1,3,Alice,3
2,2,Bob,1
3,4,Daniel,2
4,7,Luis,4
5,8,John,3
6,9,Angela,8
7,77,Robert,1


In [46]:
sql = '''with rec as (
select t.*, 0 as depth from employees t
where t.employee_id = 1
union all
select e.employee_id, e.employee_name, e.manager_id, t.depth + 1 from rec t
join employees e on t.employee_id = e.manager_id and e.employee_id != 1
limit 100
)
select * from rec t
where t.depth <= 3'''

In [47]:
select(sql)

,employee_id,employee_name,manager_id,depth
0,1,Boss,1,0
1,2,Bob,1,1
2,77,Robert,1,1
3,4,Daniel,2,2
4,7,Luis,4,3
